- 1st svm to check execution time

In [1]:
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
import os
import warnings
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm
from category_encoders import CountEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold, StratifiedKFold
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

from sklearn.kernel_approximation import Nystroem
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from cuml.svm import SVC, SVR

# preprocess

In [3]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
#non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')

In [4]:
#non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
#nontarget_dists = pd.DataFrame(np.sum(non_targets[non_target_feats])).reset_index(drop=False)
#nontarget_dists.columns = ["target", "number"]
#nontarget_dists = nontarget_dists.sort_values("number", ascending=False).reset_index(drop=True)
#drop_list1 = list(nontarget_dists[nontarget_dists.number==0]["target"].values)
#non_targets.drop(drop_list1, axis=1, inplace=True)
#print("shape after 1st drop:", non_targets.shape)

In [5]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]
#non_target_feats = [i for i in non_targets.columns if i != "sig_id"]

In [6]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

test = test[test.index.isin(cons_test_index)].reset_index(drop=True)
train = train[train.index.isin(cons_train_index)].reset_index(drop=True)
fn_targets = targets.drop("sig_id", axis=1).copy()
fn_targets = fn_targets[fn_targets.index.isin(cons_train_index)].copy().reset_index(drop=True).to_numpy()
#fn_non_targets = non_targets.drop("sig_id", axis=1).copy()
#fn_non_targets = fn_non_targets[fn_non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True).to_numpy()
y = targets.drop("sig_id", axis=1).copy()
#non_y = non_targets.drop("sig_id", axis=1).copy()

# Feature engineering

In [7]:
def fe_simple(df, remove_features):
    tmp = df.copy()
    tmp.drop(remove_features, axis=1, inplace=True)
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return tmp

remove_features = ["cp_type" , "sig_id"]
train_fn = fe_simple(train, remove_features)
test_fn = fe_simple(test, remove_features)

print(train_fn.shape, test_fn.shape)

(21948, 874) (3624, 874)


In [8]:
fn_train = train_fn.copy() 
fn_test = test_fn.copy() 

ss = preprocessing.StandardScaler()
fn_train= ss.fit_transform(fn_train)
fn_test = ss.transform(fn_test)

# modelling

In [9]:
N_STARTS = 3
N_SPLITS = 5

def modelling_svm(tr, ta, te):
    svm0_oof = np.zeros([len(tr), ta.shape[1]])
    svm0_test = np.zeros([len(te), ta.shape[1]])

    svm1_test = np.zeros([len(te),ta.shape[1]])
    svm1_oof = np.zeros([ta.shape[0],ta.shape[1]]) 

    for ind in tqdm(range(ta.shape[1])):
        ind_target_sum = ta[:, ind].sum()
        if ind_target_sum >= N_SPLITS:
            for seed in range(N_STARTS):
                skf = StratifiedKFold(n_splits = N_SPLITS, random_state = seed, shuffle = True)

                for n, (train_index, val_index) in enumerate(skf.split(tr, ta[:,ind])):
                
                    x_tr, x_val = tr[train_index], tr[val_index]
                    y_tr, y_val = ta[train_index,ind], ta[val_index,ind]

                    model = SVC(C = 40, cache_size = 2000)
                    model.fit(x_tr, y_tr)
                    svm0_test[:, ind] += model.decision_function(te) / (N_SPLITS * N_STARTS)
                    svm0_oof[val_index, ind] += model.decision_function(x_val) / N_STARTS
                    
            for seed in range(N_STARTS):
                skf = StratifiedKFold(n_splits = N_SPLITS, random_state = seed, shuffle = True)

                for n, (train_index, val_index) in enumerate(skf.split(svm0_oof, ta[:,ind])):

                    x_tr, x_val = svm0_oof[train_index, ind].reshape(-1, 1), svm0_oof[val_index, ind].reshape(-1, 1)
                    y_tr, y_val = ta[train_index,ind], ta[val_index,ind]

                    model = LogisticRegression(C = 35, max_iter = 1000)
                    model.fit(x_tr, y_tr)
                    svm1_test[:, ind] += model.predict_proba(svm0_test[:, ind].reshape(-1, 1))[:, 1] / (N_SPLITS * N_STARTS)
                    svm1_oof[val_index, ind] += model.predict_proba(x_val)[:, 1] / N_STARTS

        score1 = log_loss(ta[:, ind], svm0_oof[:, ind])
        score2 = log_loss(ta[:, ind], svm1_oof[:, ind])
        print('SVM Target ind {} score {}: {}'.format(ind, score1, score2))

    return svm1_test, svm1_oof

In [10]:
svm1_test, svm1_oof = modelling_svm(fn_train, fn_targets, fn_test)

SVM Target ind 0 score 0.0267522871657328: 0.005032248542484615
SVM Target ind 1 score 0.028325951116658196: 0.006581885570450463
SVM Target ind 2 score 0.0377679348222106: 0.008589467902867885
SVM Target ind 3 score 0.298996150675827: 0.048917698593537305
SVM Target ind 4 score 0.47367284922854636: 0.07159748761855246
SVM Target ind 5 score 0.11487746841755518: 0.022312085726263448
SVM Target ind 6 score 0.0849778533499726: 0.017221156219957106
SVM Target ind 7 score 0.15107173928883938: 0.02758612434078699
SVM Target ind 8 score 0.0188839674111058: 0.0009619483301645337
SVM Target ind 9 score 0.41774788362703535: 0.06028960923052172
SVM Target ind 10 score 0.566519022333145: 0.08282931815972802
SVM Target ind 11 score 0.07872803928766862: 0.011506857753246792
SVM Target ind 12 score 0.0110156476564788: 0.000490875148437014
SVM Target ind 13 score 0.06609388593886781: 0.009981667824920612
SVM Target ind 14 score 0.0188839674111058: 0.004699142468417396
SVM Target ind 15 score 0.018883

In [11]:
check_svm1 = np.zeros([targets.shape[0], targets.shape[1]-1])
check_svm1[cons_train_index,:] = svm1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_svm1)))

#check_non_svm1 = np.zeros([non_targets.shape[0], non_targets.shape[1]-1])
#check_non_svm1[cons_train_index,:] = svm1_non_oof
#print('OOF log loss: ', log_loss(np.ravel(non_y), np.ravel(check_non_svm1)))

OOF log loss:  0.015288382527672173


# submission

In [12]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

sub.loc[cons_test_index,target_feats] =  svm1_test 
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)